In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import random
import numpy as np
import cv2 as cv
import os
import shutil


In [3]:
def sp_noise(image,prob):
    '''
    Add salt and pepper noise to image
    prob: Probability of the noise
    '''
    output = np.zeros(image.shape,np.uint8)
    thres = 1 - prob 
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rdn = random.random()
            if rdn < prob:
                output[i][j] = 0
            elif rdn > thres:
                output[i][j] = 255
            else:
                output[i][j] = image[i][j]
    return output


def add_noise(image):
        im = np.zeros(image.shape, np.uint8) # do not use original image it overwrites the image
        mean = 0
        sigma = random.randint(2,10)
        cv.randn(im,mean,sigma) # create the random distribution
        noisy = cv.add(image, im) # add the noise to the original image
        return noisy
        
def add_blur(image):
        im = np.zeros(image.shape, np.uint8)
        mean = 0
        sigma = random.randint(2,10)
        noise = cv.randn(im,mean,sigma) 
        noise_image = cv.addWeighted(image, 0.5, noise, 0.5, 50) 
        blurred_image = cv.GaussianBlur(noise_image, (15, 15), 0)
        return blurred_image

In [12]:
annotation_path = 'drive/MyDrive/szlaban/plate_training/dataset/annotations_big/'
images_path = 'drive/MyDrive/szlaban/plate_training/dataset/images_big/'


In [13]:
for file in os.listdir(images_path):
    object_name = file.replace(".png", "")
    annotation_file = f'{annotation_path}/{object_name}.xml'
    image_file = f'{images_path}/{file}'
    
    opencv_image = cv.imread(image_file)
    opencv_gray_image = cv.cvtColor(opencv_image, cv.COLOR_BGR2GRAY)

    '''
    augumenting parameters:
    gaussian blur  cv.GaussianBlur(img,(5,5),0)
    noise          sp_noise(image, prob)
    
    method:
    1 clean grayscale/color image
    3 grayscale image with sp noise
    3 grayscale/color image with noise
    3 grayscale/color image with gaussianBlur
    3 grayscale/color image with random parameters
    '''
    
    #augumenting grayscale_image
    
    #1 - save clean grayscale image and annotation_file for it
    grayscale_image_name = f'{object_name}_gray.png'
    cv.imwrite(f'{images_path}/{grayscale_image_name}', opencv_gray_image)
    shutil.copy(annotation_file, f'{annotation_path}/{object_name}_gray.xml')
    
    #2 - add some sp noise
    for i in range (0,3):
        prob = random.uniform(0.01, 0.1)
        noisy_sp_image = sp_noise(opencv_gray_image, prob)
        noisy_sp_image_name = f'{object_name}_noisy_sp_{i}.png'
        cv.imwrite(f'{images_path}/{noisy_sp_image_name}', noisy_sp_image)
        shutil.copy(annotation_file, f'{annotation_path}/{object_name}_noisy_sp_{i}.xml')
        
    #3 - add some noise for gray and color images:
    for i in range(0,3):
        #gray
        noisy_gray_image = add_noise(opencv_gray_image)
        noisy_gray_image_name = f'{object_name}_gray_noisy_{i}.png'
        #color
        noisy_color_image = add_noise(opencv_image)
        noisy_color_image_name = f'{object_name}_color_noisy_{i}.png'
        
        #gray
        cv.imwrite(f'{images_path}/{noisy_gray_image_name}', noisy_gray_image)
        shutil.copy(annotation_file, f'{annotation_path}/{object_name}_gray_noisy_{i}.xml')
        #color
        cv.imwrite(f'{images_path}/{noisy_color_image_name}', noisy_color_image)
        shutil.copy(annotation_file, f'{annotation_path}/{object_name}_color_noisy_{i}.xml')
        
        
    #3 - add some gaussianBlur for gray and color images:
    for i in range(0,3):
        #gray
        blurred_gray_image = add_blur(opencv_gray_image)
        blurred_gray_image_name = f'{object_name}_gray_blurred_{i}.png'
        #color
        blurred_color_image = add_blur(opencv_image)
        blurred_color_image_name = f'{object_name}_color_blurred_{i}.png'
        
        #gray
        cv.imwrite(f'{images_path}/{blurred_gray_image_name}', blurred_gray_image)
        shutil.copy(annotation_file, f'{annotation_path}/{object_name}_gray_blurred_{i}.xml')
        #color
        cv.imwrite(f'{images_path}/{blurred_color_image_name}', blurred_color_image)
        shutil.copy(annotation_file, f'{annotation_path}/{object_name}_color_blurred_{i}.xml')
        
    #4 - add mixed parameters
    for i in range(0,3):
        #gray
        final_image = opencv_gray_image
        #sp?
        if random.random() >= 0.5:  final_image = sp_noise(final_image, 0.03)
        #gaussian?
        if random.random() >= 0.5:  final_image = add_blur(final_image)
        #noise?
        if random.random() >= 0.5:  final_image = add_noise(final_image)
        
        mixed_gray_file_name = f'{object_name}_gray_mixed_{i}.png'
        cv.imwrite(f'{images_path}/{mixed_gray_file_name}', final_image)
        shutil.copy(annotation_file, f'{annotation_path}/{object_name}_gray_mixed_{i}.xml')
        
        #color
        final_image = opencv_image
        #gaussian?
        if random.random() >= 0.5:  final_image = add_blur(final_image)
        #noise?
        if random.random() >= 0.5:  final_image = add_noise(final_image)
        
        mixed_color_file_name = f'{object_name}_color_mixed_{i}.png'
        cv.imwrite(f'{images_path}/{mixed_color_file_name}', final_image)
        shutil.copy(annotation_file, f'{annotation_path}/{object_name}_color_mixed_{i}.xml')

In [21]:
!ls drive/MyDrive/szlaban/plate_training/dataset/annotations_big

Cars0_color_blurred_0.xml    Cars295_gray_mixed_0.xml
Cars0_color_blurred_1.xml    Cars295_gray_mixed_1.xml
Cars0_color_blurred_2.xml    Cars295_gray_mixed_2.xml
Cars0_color_mixed_0.xml      Cars295_gray_noisy_0.xml
Cars0_color_mixed_1.xml      Cars295_gray_noisy_1.xml
Cars0_color_mixed_2.xml      Cars295_gray_noisy_2.xml
Cars0_color_noisy_0.xml      Cars295_gray.xml
Cars0_color_noisy_1.xml      Cars295_noisy_sp_0.xml
Cars0_color_noisy_2.xml      Cars295_noisy_sp_1.xml
Cars0_gray_blurred_0.xml     Cars295_noisy_sp_2.xml
Cars0_gray_blurred_1.xml     Cars295.xml
Cars0_gray_blurred_2.xml     Cars296_color_blurred_0.xml
Cars0_gray_mixed_0.xml	     Cars296_color_blurred_1.xml
Cars0_gray_mixed_1.xml	     Cars296_color_blurred_2.xml
Cars0_gray_mixed_2.xml	     Cars296_color_mixed_0.xml
Cars0_gray_noisy_0.xml	     Cars296_color_mixed_1.xml
Cars0_gray_noisy_1.xml	     Cars296_color_mixed_2.xml
Cars0_gray_noisy_2.xml	     Cars296_color_noisy_0.xml
Cars0_gray.xml		     Cars296_color_noisy_1.xml
C